In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import math as m
warnings.filterwarnings('ignore')

In [197]:
df = pd.read_csv('total.csv')

In [198]:
len(df)

15109

In [199]:
df = df.drop_duplicates()
len(df)

15109

In [200]:
df['racedater'] =  pd.to_datetime(df['racedater'], format='%Y-%m-%d')
df.dtypes

racedater                     datetime64[ns]
tvgtrackcode                          object
race                                   int64
bettinginterestnumber                  int64
horsename                             object
morninglineodds                      float64
finalodds                             object
tvghorseweight                         int64
tvghorsedamsirename                   object
tvghorseage                            int64
tvghorsesex                           object
tvghorsedaysoff                        int64
tvghorsenumberofwins                   int64
tvghorsenumberofstarts                 int64
tvghorsepowerrating                  float64
tvghorseaveragespeed                   int64
tvghorseaverageclassrating             int64
currentodds                           object
winpayout                            float64
placepayout                          float64
showpayout                           float64
scratched                              int64
won       

In [201]:
df.sort_values('racedater').head()

,racedater,tvgtrackcode,race,bettinginterestnumber,horsename,morninglineodds,finalodds,tvghorseweight,tvghorsedamsirename,tvghorseage,...,tvghorsepowerrating,tvghorseaveragespeed,tvghorseaverageclassrating,currentodds,winpayout,placepayout,showpayout,scratched,won,currentodds.1
52,2018-05-01,cd,5,5,show me indy,12.0,13,118,albert the great,6,...,68.0,63,73,13,0.0,0.0,0.0,0,0,NaN
36,2018-05-01,cd,3,6,drop kick,2.5,2,118,royal academy,3,...,61.4,66,78,2,6.2,3.2,2.6,0,1,NaN
37,2018-05-01,cd,3,5,granian,3.5,3,118,unbridled's song,3,...,92.3,68,78,3,0.0,0.0,0.0,0,0,NaN
38,2018-05-01,cd,3,4,fearless kitty,3.5,5,118,forest wildcat,3,...,74.3,72,72,5,0.0,0.0,0.0,0,0,NaN
39,2018-05-01,cd,3,2,southeast,3.0,3,118,street sense,3,...,76.0,74,84,3,0.0,3.8,2.8,0,0,NaN


In [202]:
features = [
    'tvghorseweight', 
    'tvghorseage', 
    'tvghorsedaysoff', 
    'tvghorsenumberofwins', 
    'tvghorsepowerrating', 
    'tvghorseaverageclassrating',
    'tvghorseaveragespeed'
]
target = 'won'

In [203]:
len(df)

15109

In [204]:
from datetime import datetime as dt
dt(2018,5,2).strftime("%Y-%m-%d")

'2018-05-02'

In [213]:
from datetime import datetime as dt
start_date = dt(2018,5,2).strftime("%Y-%m-%d")
end_date = dt(2020,5,31).strftime("%Y-%m-%d")
train = df[(df['racedater'] > start_date) & (df['racedater'] <= end_date)]
train = train.sort_values('racedater')
print(len(train))
train = train.drop_duplicates()
train

12694


,racedater,tvgtrackcode,race,bettinginterestnumber,horsename,morninglineodds,finalodds,tvghorseweight,tvghorsedamsirename,tvghorseage,...,tvghorsepowerrating,tvghorseaveragespeed,tvghorseaverageclassrating,currentodds,winpayout,placepayout,showpayout,scratched,won,currentodds.1
66,2018-05-03,cd,1,6,toothpic,3.0,3.5,118,housebuster,3,...,75.7,69,82,7/2,0.0,0.0,2.2,0,0,NaN
253,2018-05-03,cd,8,8,closing bell,8.0,6,122,coronado's quest,6,...,84.9,93,106,6,0.0,0.0,0.0,0,0,NaN
252,2018-05-03,cd,8,9,royal son,12.0,8,122,a.p. indy,6,...,115.0,106,107,8,0.0,0.0,0.0,0,0,NaN
251,2018-05-03,cd,7,2,big coulee,15.0,9,118,bates motel,5,...,73.5,76,89,9,0.0,0.0,0.0,0,0,NaN
250,2018-05-03,cd,7,3,sunny's rainbow,3.5,2.5,111,gilded time,5,...,94.5,90,96,5/2,0.0,0.0,0.0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14029,2020-05-31,cd,6,5,worthaprettypenny,8.0,NaN,118,afternoon deelites,3,...,49.7,45,74,18,0.0,0.0,0.0,0,0,18
14028,2020-05-31,cd,6,2,miss annie,15.0,NaN,124,storm and a half,4,...,35.1,34,66,25,0.0,0.0,0.0,0,0,25
13362,2020-05-31,cd,5,9,jeweled princess,3.5,4,118,scat daddy,3,...,84.2,75,87,4,0.0,0.0,3.4,0,0,NaN
13360,2020-05-31,cd,5,7,free,8.0,13,118,double honor,3,...,70.5,70,75,13,0.0,0.0,0.0,0,0,NaN


In [215]:
test = df[(df['racedater'] > end_date)]
test = test.sort_values(['racedater','race'])
print(len(test))
test = test.drop_duplicates()
len(test)

2270


2270

In [86]:
X_train = train[features]
y_train = train[target].values.reshape(-1,1)

In [87]:
X_test = test[features]
y_test = test[target].values.reshape(-1,1)

# Machine Learning Models

## SVM

### Training the SVM model and finding a score

In [127]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, balanced_accuracy_score, average_precision_score, brier_score_loss

model_svm = SVC(kernel='sigmoid')
model_svm.fit(X_train, y_train)

train_score_svm = model_svm.score(X_train, y_train)
test_score_svm = model_svm.score(X_test, y_test)

y_pred_svm = model_svm.predict(X_test)

accuracy_score_svm = accuracy_score(y_test, y_pred_svm)
bal_acc_score_svm = balanced_accuracy_score(y_test, predictions_svm)
avg_prec_score_svm = average_precision_score(y_test, predictions_svm)


print(f'''
-----------------------------------
|Training R\u00b2 Score:        |{100*round(train_score_svm,4)}%|
-----------------------------------
|Test R\u00b2 Score:            |{100*round(test_score_svm,4)}%|
-----------------------------------
|Predicted accuracy score: |{100*round(accuracy_score_svm,4)}%|
-----------------------------------
|Balanced accuracy score:  |{100*round(bal_acc_score_svm,4)}%|
-----------------------------------
|Avg precision score:      |{100*round(avg_prec_score_svm,4)}%|
-----------------------------------
''')



-----------------------------------
|Training R² Score:        |79.42%|
-----------------------------------
|Test R² Score:            |84.58%|
-----------------------------------
|Predicted accuracy score: |84.58%|
-----------------------------------
|Balanced accuracy score:  |48.39%|
-----------------------------------
|Avg precision score:      |11.25%|
-----------------------------------



### SVM Classification Report

In [7]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_svm,
                            target_names=["won", "lost"]))

              precision    recall  f1-score   support

         win       0.89      0.89      0.89      1410
        lost       0.11      0.11      0.11       177

    accuracy                           0.80      1587
   macro avg       0.50      0.50      0.50      1587
weighted avg       0.80      0.80      0.80      1587



## XGBoost

### Training the XGBoost Classifier and finding a score

In [148]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, average_precision_score, r2_score

model_XGB = XGBClassifier()
model_XGB.fit(X_train, y_train)


y_pred_XGB = model_XGB.predict(X_test)

r2_score_XGB = r2_score(y_test, y_pred_XGB)
accuracy_score_XGB = accuracy_score(y_test, y_pred_XGB)
bal_acc_score_XGB = balanced_accuracy_score(y_test, y_pred_XGB)
avg_prec_score_XGB = average_precision_score(y_test, y_pred_XGB)


print(f'''
R\u00b2 Score:            {100*round(r2_score_XGB,4)}%

Predicted accuracy score: {100*round(accuracy_score_XGB,4)}%

Balanced accuracy score:  {100*round(bal_acc_score_XGB,4)}%

Avg precision score:      {100*round(avg_prec_score_XGB,4)}%

''')


R² Score:            -15.45%

Predicted accuracy score: 88.37%

Balanced accuracy score:  51.2%

Avg precision score:      12.139999999999999%




### XGBoost Classification Report

In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_XGB,
                            target_names=["won", "lost"]))

              precision    recall  f1-score   support

         win       0.90      1.00      0.95      1410
        lost       1.00      0.14      0.25       177

    accuracy                           0.90      1587
   macro avg       0.95      0.57      0.60      1587
weighted avg       0.91      0.90      0.87      1587



## Naive-Bayes Gaussian Classifier

### Training the Naive-Bayes Gaussian model and finding a score

In [108]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, balanced_accuracy_score, average_precision_score


model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

train_score_nb = model_nb.score(X_train, y_train)
test_score_nb = model_nb.score(X_test, y_test)

y_pred_nb = model_nb.predict(X_test)

accuracy_score_nb = accuracy_score(y_test, y_pred_nb)
bal_acc_score_nb = balanced_accuracy_score(y_test, y_pred_nb)
avg_prec_score_nb = average_precision_score(y_test, y_pred_nb)

print(f'''
R\u00b2 Scores
-----------------------
Training Score:         {100*round(train_score_nb,4)}%
Test Score:             {100*round(test_score_nb,4)}%
Accuracy Score:         {100*round(accuracy_score_nb,4)}%
Balance Accuracy Score: {100*round(avg_prec_score_nb,4)}%
''')


R² Scores
-----------------------
Training Score:         87.58%
Test Score:             87.83999999999999%
Accuracy Score:         87.83999999999999%
Balance Accuracy Score: 11.43%



### Naive-Bayes Classification Report

In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_nb,
                            target_names=["win", "lost"]))

              precision    recall  f1-score   support

         win       0.89      0.99      0.93      1410
        lost       0.16      0.02      0.04       177

    accuracy                           0.88      1587
   macro avg       0.52      0.50      0.49      1587
weighted avg       0.81      0.88      0.83      1587



# Exporting the results as DataFrames

In [170]:
test_copy = test.copy()

In [171]:
test_copy.columns

Index(['racedater', 'tvgtrackcode', 'race', 'bettinginterestnumber',
       'horsename', 'morninglineodds', 'finalodds', 'tvghorseweight',
       'tvghorsedamsirename', 'tvghorseage', 'tvghorsesex', 'tvghorsedaysoff',
       'tvghorsenumberofwins', 'tvghorsenumberofstarts', 'tvghorsepowerrating',
       'tvghorseaveragespeed', 'tvghorseaverageclassrating', 'currentodds',
       'winpayout', 'placepayout', 'showpayout', 'scratched', 'won',
       'currentodds.1'],
      dtype='object')

In [172]:
cols = [
    'racedater',
    'horsename',
    'tvghorseweight', 
    'tvghorseage', 
    'tvghorsedaysoff', 
    'tvghorsenumberofwins', 
    'tvghorsepowerrating', 
    'tvghorseaverageclassrating',
    'tvghorseaveragespeed',
    'won'
]
test_copy = test_copy[cols]
test_copy.head()

,racedater,horsename,tvghorseweight,tvghorseage,tvghorsedaysoff,tvghorsenumberofwins,tvghorsepowerrating,tvghorseaverageclassrating,tvghorseaveragespeed,won
12656,2020-06-04,french rose,118,3,18,2,74.0,84,70,0
12657,2020-06-04,foreign exchange,120,3,32,2,76.4,83,66,0
12658,2020-06-04,trafficiskillingme,118,3,92,0,70.2,72,66,0
12659,2020-06-04,dehydration,118,3,55,1,71.4,74,63,1
12660,2020-06-04,heart river,118,3,17,1,62.8,72,63,0


In [173]:
cols_replace = {
    'racedater':'Date',
    'horsename':'Horse Name',
    'tvghorseweight':'Horse Weight', 
    'tvghorseage':'Horse Age', 
    'tvghorsedaysoff':'Days Off', 
    'tvghorsenumberofwins':'Number of Wins', 
    'tvghorsepowerrating':'Power Rating', 
    'tvghorseaverageclassrating':'Average Class Rating',
    'tvghorseaveragespeed':'Average Speed',
    'won':'Won Actual'
}

In [174]:
test_copy.rename(columns = cols_replace, inplace=True)
test_copy.tail()

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual
14269,2020-06-27,crystal cliffs (fr),118,3,26,1,94.9,90,97,0
14270,2020-06-27,micheline,118,3,55,3,89.3,88,78,0
14271,2020-06-27,edgee angel,118,3,40,0,75.3,80,66,0
14272,2020-06-27,wexx,118,3,33,1,90.9,87,82,0
14273,2020-06-27,eve of war,118,3,34,1,95.6,91,89,0


In [176]:
test_copy['Won SVM'] = predictions_svm
test_copy['Won XGBoost'] = predictions_XGB
test_copy['Won Naive-Bayes'] = predictions_nb
test_copy

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12656,2020-06-04,french rose,118,3,18,2,74.0,84,70,0,0,0,0
12657,2020-06-04,foreign exchange,120,3,32,2,76.4,83,66,0,0,0,0
12658,2020-06-04,trafficiskillingme,118,3,92,0,70.2,72,66,0,0,0,0
12659,2020-06-04,dehydration,118,3,55,1,71.4,74,63,1,0,0,0
12660,2020-06-04,heart river,118,3,17,1,62.8,72,63,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14269,2020-06-27,crystal cliffs (fr),118,3,26,1,94.9,90,97,0,0,0,0
14270,2020-06-27,micheline,118,3,55,3,89.3,88,78,0,0,0,0
14271,2020-06-27,edgee angel,118,3,40,0,75.3,80,66,0,0,0,0
14272,2020-06-27,wexx,118,3,33,1,90.9,87,82,0,0,0,0


In [ ]:
# Create dataframes for each ML model used following

# mw == machine won
# ml == machine lost
# aw == actual won
# al == actual lost

# MW | MW
# AW | AL
# ------
# ML| ML
# AW | AL

In [232]:
svm_mw_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won SVM'] == 1)].drop_duplicates()
svm_mw_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won SVM'] == 1)].drop_duplicates()
svm_ml_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won SVM'] == 0)].drop_duplicates()
svm_ml_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won SVM'] == 0)].drop_duplicates()

xgb_mw_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won XGBoost'] == 1)].drop_duplicates()
xgb_mw_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won XGBoost'] == 1)].drop_duplicates()
xgb_ml_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won XGBoost'] == 0)].drop_duplicates()
xgb_ml_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won XGBoost'] == 0)].drop_duplicates()

nb_mw_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won Naive-Bayes'] == 1)].drop_duplicates()
nb_mw_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won Naive-Bayes'] == 1)].drop_duplicates()
nb_ml_aw = test_copy.loc[(test_copy['Won Actual'] == 1) & (test_copy['Won Naive-Bayes'] == 0)].drop_duplicates()
nb_ml_al = test_copy.loc[(test_copy['Won Actual'] == 0) & (test_copy['Won Naive-Bayes'] == 0)].drop_duplicates()

In [233]:
svm_mw_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12221,2020-06-05,orb's soul,118,3,18,0,59.6,70,60,1,1,0,0
13454,2020-06-18,jamin's jet,120,2,23,0,59.2,74,52,1,1,0,0


In [234]:
svm_mw_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12660,2020-06-04,heart river,118,3,17,1,62.8,72,63,0,1,0,0
12672,2020-06-04,bionic rosie,118,3,77,0,38.7,62,35,0,1,0,0
12722,2020-06-04,baquero flies,118,3,18,1,55.7,77,55,0,1,0,0
12095,2020-06-05,mi chachis intenta,118,3,18,1,66.4,73,60,0,1,0,0
12098,2020-06-05,red jersey,123,4,18,1,39.6,77,52,0,1,0,0
12108,2020-06-05,twinkling knight,118,6,19,1,63.1,78,58,0,1,0,0
12763,2020-06-07,nomiraclesneeded,118,3,91,0,29.9,60,22,0,1,0,0
8973,2020-06-11,summer war,119,2,20,0,42.9,83,36,0,1,0,0
8974,2020-06-11,little empire,119,2,16,0,45.9,74,33,0,1,0,0
8759,2020-06-11,yamato,118,3,104,0,38.7,59,32,0,1,0,0


In [235]:
svm_ml_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12659,2020-06-04,dehydration,118,3,55,1,71.4,74,63,1,0,0,0
12664,2020-06-04,lord dragon,124,4,33,1,77.1,95,84,1,0,0,0
12673,2020-06-04,flashy biz,125,4,34,0,58.5,78,67,1,0,0,0
12719,2020-06-04,silver ride,118,8,39,7,100.2,104,96,1,0,0,0
12727,2020-06-04,botero,123,5,41,1,82.3,96,81,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13277,2020-06-27,wellabled,121,6,28,0,88.1,109,103,1,0,0,0
13286,2020-06-27,cazadero,120,2,28,1,104.2,87,99,1,0,1,0
13292,2020-06-27,midnight bisou,124,5,118,11,107.2,112,111,1,0,1,1
13330,2020-06-27,tom's d'etat,124,7,76,10,111.2,115,109,1,0,0,0


In [236]:
svm_ml_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12656,2020-06-04,french rose,118,3,18,2,74.0,84,70,0,0,0,0
12657,2020-06-04,foreign exchange,120,3,32,2,76.4,83,66,0,0,0,0
12658,2020-06-04,trafficiskillingme,118,3,92,0,70.2,72,66,0,0,0,0
12661,2020-06-04,skyvalue,120,3,46,1,85.9,73,77,0,0,0,0
12662,2020-06-04,urbanite,118,3,41,1,78.8,90,74,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13342,2020-06-27,crystal cliffs (fr),118,3,26,1,94.9,90,97,0,0,0,0
13343,2020-06-27,micheline,118,3,55,3,89.3,88,78,0,0,0,0
13344,2020-06-27,edgee angel,118,3,40,0,75.3,80,66,0,0,0,0
13345,2020-06-27,wexx,118,3,33,1,90.9,87,82,0,0,0,0


In [237]:
xgb_mw_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12849,2020-06-07,lead guitar,123,4,232,1,84.2,86,89,1,0,1,0
13268,2020-06-27,paris lights,120,3,26,1,110.4,85,92,1,0,1,0
13286,2020-06-27,cazadero,120,2,28,1,104.2,87,99,1,0,1,0
13292,2020-06-27,midnight bisou,124,5,118,11,107.2,112,111,1,0,1,1


In [238]:
xgb_mw_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
13051,2020-06-06,america's surprise,118,3,299,0,76.3,88,83,0,0,1,0
13068,2020-06-06,tohui,123,4,314,1,83.8,88,89,0,0,1,0
12753,2020-06-07,long on luck,118,6,476,3,74.0,104,92,0,0,1,0
13451,2020-06-13,lombo,120,5,41,0,78.5,108,95,0,0,1,0
11858,2020-06-20,hackberry,124,4,230,1,93.8,101,97,0,0,1,0
12514,2020-06-21,the rock says,124,4,233,1,87.8,100,98,0,0,1,0
12500,2020-06-21,joy epifora (arg),122,4,195,1,93.5,99,97,0,0,1,0


In [239]:
xgb_ml_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12659,2020-06-04,dehydration,118,3,55,1,71.4,74,63,1,0,0,0
12664,2020-06-04,lord dragon,124,4,33,1,77.1,95,84,1,0,0,0
12673,2020-06-04,flashy biz,125,4,34,0,58.5,78,67,1,0,0,0
12719,2020-06-04,silver ride,118,8,39,7,100.2,104,96,1,0,0,0
12727,2020-06-04,botero,123,5,41,1,82.3,96,81,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13180,2020-06-27,hemp hemp hurray,118,5,97,0,72.9,101,86,1,0,0,0
13260,2020-06-27,amani's eagle,118,3,41,0,84.8,90,80,1,0,0,0
13277,2020-06-27,wellabled,121,6,28,0,88.1,109,103,1,0,0,0
13330,2020-06-27,tom's d'etat,124,7,76,10,111.2,115,109,1,0,0,0


In [240]:
xgb_ml_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12656,2020-06-04,french rose,118,3,18,2,74.0,84,70,0,0,0,0
12657,2020-06-04,foreign exchange,120,3,32,2,76.4,83,66,0,0,0,0
12658,2020-06-04,trafficiskillingme,118,3,92,0,70.2,72,66,0,0,0,0
12660,2020-06-04,heart river,118,3,17,1,62.8,72,63,0,1,0,0
12661,2020-06-04,skyvalue,120,3,46,1,85.9,73,77,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13342,2020-06-27,crystal cliffs (fr),118,3,26,1,94.9,90,97,0,0,0,0
13343,2020-06-27,micheline,118,3,55,3,89.3,88,78,0,0,0,0
13344,2020-06-27,edgee angel,118,3,40,0,75.3,80,66,0,0,0,0
13345,2020-06-27,wexx,118,3,33,1,90.9,87,82,0,0,0,0


In [241]:
nb_mw_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
13171,2020-06-27,frost or frippery,123,7,36,12,92.0,95,92,1,0,0,1
13292,2020-06-27,midnight bisou,124,5,118,11,107.2,112,111,1,0,1,1


In [242]:
nb_mw_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12714,2020-06-04,stay home,118,6,13,9,89.1,96,83,0,0,0,1
12968,2020-06-07,flowers for lisa,122,7,175,12,83.6,108,88,0,0,0,1
8840,2020-06-11,pat daddy,120,8,28,10,92.5,100,93,0,0,0,1
13401,2020-06-13,degrom,118,6,42,9,92.3,102,95,0,0,0,1
13403,2020-06-13,hardly a secret,118,6,21,9,89.5,100,84,0,0,0,1
13404,2020-06-13,popular kid,118,6,41,9,97.4,104,93,0,0,0,1
12901,2020-06-19,morticia,121,6,21,10,106.3,108,103,0,0,0,1
12397,2020-06-21,pete's play call,121,7,27,10,96.8,104,95,0,0,0,1
12691,2020-06-25,arthur's hope,121,6,128,12,101.7,103,91,0,0,0,1
12797,2020-06-26,mr. misunderstood,122,6,33,12,102.2,110,97,0,0,0,1


In [243]:
nb_ml_aw

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12659,2020-06-04,dehydration,118,3,55,1,71.4,74,63,1,0,0,0
12664,2020-06-04,lord dragon,124,4,33,1,77.1,95,84,1,0,0,0
12673,2020-06-04,flashy biz,125,4,34,0,58.5,78,67,1,0,0,0
12719,2020-06-04,silver ride,118,8,39,7,100.2,104,96,1,0,0,0
12727,2020-06-04,botero,123,5,41,1,82.3,96,81,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13268,2020-06-27,paris lights,120,3,26,1,110.4,85,92,1,0,1,0
13277,2020-06-27,wellabled,121,6,28,0,88.1,109,103,1,0,0,0
13286,2020-06-27,cazadero,120,2,28,1,104.2,87,99,1,0,1,0
13330,2020-06-27,tom's d'etat,124,7,76,10,111.2,115,109,1,0,0,0


In [244]:
nb_ml_al

,Date,Horse Name,Horse Weight,Horse Age,Days Off,Number of Wins,Power Rating,Average Class Rating,Average Speed,Won Actual,Won SVM,Won XGBoost,Won Naive-Bayes
12656,2020-06-04,french rose,118,3,18,2,74.0,84,70,0,0,0,0
12657,2020-06-04,foreign exchange,120,3,32,2,76.4,83,66,0,0,0,0
12658,2020-06-04,trafficiskillingme,118,3,92,0,70.2,72,66,0,0,0,0
12660,2020-06-04,heart river,118,3,17,1,62.8,72,63,0,1,0,0
12661,2020-06-04,skyvalue,120,3,46,1,85.9,73,77,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13342,2020-06-27,crystal cliffs (fr),118,3,26,1,94.9,90,97,0,0,0,0
13343,2020-06-27,micheline,118,3,55,3,89.3,88,78,0,0,0,0
13344,2020-06-27,edgee angel,118,3,40,0,75.3,80,66,0,0,0,0
13345,2020-06-27,wexx,118,3,33,1,90.9,87,82,0,0,0,0
